In [1]:
import sys
import os
import mido

In [2]:
from mido import MidiFile
import os
import sys
from torch.utils.data import Dataset, DataLoader

In [3]:
ourmidi = MidiFile('../clean_midi/Ace of Base/The Sign.2.mid')

In [4]:
for i, track in enumerate(ourmidi.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg.type)

Track 0: 
time_signature
sequencer_specific
set_tempo
end_of_track
Track 1: Melody
track_name
control_change
program_change
key_signature
control_change
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_o

note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_on
note_off
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_o

note_on
note_off
note_on
note_off
end_of_track
Track 19: Crash Cymbal
track_name
control_change
program_change
key_signature
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
end_of_track
Track 20: Timbale
track_name
control_change
program_change
key_signature
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_on
note_off
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on

In [5]:
import torch
import torch.nn.functional as functional

class MIDITrackError(Exception):
    pass

class MessageSequence:
    def __init__(self, mid):
        self.messages = []
        try: 
            for i in mid.tracks[1]:
                #print(i.type)
                if i.type in ['note_on', 'note_off']:
                    self.messages.append(i)
        except IndexError:
            raise MIDITrackError
        
        #calculate note durations
        timecounter = 0
        notedict = {}
        real_sequence = []
        for message in self.messages:
            timecounter += message.time
            if message.type == "note_on":
                notedict[message.note] = timecounter
                
            if message.type == "note_off":
                duration = timecounter - notedict[message.note]
                real_sequence.append((message.note, notedict[message.note], message.time, duration))
                
        self.sequence = real_sequence
        
    def midi_reencode(self):
        reencoded = []
        active_notes = {}
        timecounter = 0
        for (note, timestamp, offset, duration) in self.sequence:
            note_order = sorted(active_notes.keys(), key=lambda x: active_notes[x][0]) #timestamp is tuple item 0
            for active_note in note_order:
                if active_notes[active_note][0] < timestamp:
                    reencoded.append(mido.Message("note_off", 
                                                  channel=1, 
                                                  note=active_note, 
                                                  velocity=95, 
                                                  time=active_notes[active_note][1]))
                    timecounter += active_notes[active_note][1]
                    del active_notes[active_note]
            reencoded.append(mido.Message("note_on", 
                                          channel=1, 
                                          note=note,
                                          velocity=95, 
                                          time=timestamp-timecounter))
            active_notes[note] = (timestamp+duration, offset, duration)
            timecounter = timestamp

            
        note_order = sorted(active_notes.keys(), key=lambda x: active_notes[x][0]) #timestamp is tuple item 0
        for active_note in note_order:
            reencoded.append(mido.Message("note_off", 
                                          channel=1, 
                                          note=active_note, 
                                          velocity=95, 
                                          time=active_notes[active_note][1]))
            del active_notes[active_note]
            
        return reencoded
    
    def vector_encode(self):
        note_db = functional.one_hot(torch.arange(0, 128))
        encoded = []
        for (note, _, offset, duration) in self.sequence:
            note_vec = note_db[note].clone().detach()
            if offset > 100:
                offset = 100
            if duration > 4000:
                duration = 4000
            
            offset = offset/100
            duration = duration/4000
            
            encoded.append(torch.cat((note_vec, torch.Tensor([offset]), torch.Tensor([duration]))))
        return encoded

In [6]:
x = MessageSequence(ourmidi)

In [7]:
x.messages, x.sequence

([Message('note_on', channel=0, note=59, velocity=100, time=20352),
  Message('note_off', channel=0, note=59, velocity=0, time=96),
  Message('note_on', channel=0, note=59, velocity=100, time=0),
  Message('note_off', channel=0, note=59, velocity=0, time=96),
  Message('note_on', channel=0, note=62, velocity=100, time=0),
  Message('note_off', channel=0, note=62, velocity=0, time=96),
  Message('note_on', channel=0, note=66, velocity=100, time=0),
  Message('note_off', channel=0, note=66, velocity=0, time=288),
  Message('note_on', channel=0, note=67, velocity=100, time=0),
  Message('note_off', channel=0, note=67, velocity=0, time=192),
  Message('note_on', channel=0, note=67, velocity=100, time=192),
  Message('note_off', channel=0, note=67, velocity=0, time=96),
  Message('note_on', channel=0, note=67, velocity=100, time=0),
  Message('note_off', channel=0, note=67, velocity=0, time=96),
  Message('note_on', channel=0, note=67, velocity=100, time=0),
  Message('note_off', channel=0,

In [8]:
x.midi_reencode()

[Message('note_on', channel=1, note=59, velocity=95, time=20352),
 Message('note_on', channel=1, note=59, velocity=95, time=96),
 Message('note_on', channel=1, note=62, velocity=95, time=96),
 Message('note_off', channel=1, note=59, velocity=95, time=96),
 Message('note_on', channel=1, note=66, velocity=95, time=0),
 Message('note_off', channel=1, note=62, velocity=95, time=96),
 Message('note_on', channel=1, note=67, velocity=95, time=192),
 Message('note_off', channel=1, note=66, velocity=95, time=288),
 Message('note_off', channel=1, note=67, velocity=95, time=192),
 Message('note_on', channel=1, note=67, velocity=95, time=-96),
 Message('note_on', channel=1, note=67, velocity=95, time=96),
 Message('note_on', channel=1, note=67, velocity=95, time=96),
 Message('note_off', channel=1, note=67, velocity=95, time=96),
 Message('note_on', channel=1, note=67, velocity=95, time=96),
 Message('note_off', channel=1, note=67, velocity=95, time=96),
 Message('note_on', channel=1, note=69, vel

In [9]:
x.messages

[Message('note_on', channel=0, note=59, velocity=100, time=20352),
 Message('note_off', channel=0, note=59, velocity=0, time=96),
 Message('note_on', channel=0, note=59, velocity=100, time=0),
 Message('note_off', channel=0, note=59, velocity=0, time=96),
 Message('note_on', channel=0, note=62, velocity=100, time=0),
 Message('note_off', channel=0, note=62, velocity=0, time=96),
 Message('note_on', channel=0, note=66, velocity=100, time=0),
 Message('note_off', channel=0, note=66, velocity=0, time=288),
 Message('note_on', channel=0, note=67, velocity=100, time=0),
 Message('note_off', channel=0, note=67, velocity=0, time=192),
 Message('note_on', channel=0, note=67, velocity=100, time=192),
 Message('note_off', channel=0, note=67, velocity=0, time=96),
 Message('note_on', channel=0, note=67, velocity=100, time=0),
 Message('note_off', channel=0, note=67, velocity=0, time=96),
 Message('note_on', channel=0, note=67, velocity=100, time=0),
 Message('note_off', channel=0, note=67, veloci

In [10]:
x.vector_encode()

[tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [11]:
import torch.nn as nn

In [12]:
import torch

In [13]:
import torch.optim as optim

In [14]:
class MIDIModel(nn.Module):
    def __init__(self, hidden_layer_size=64):
        super().__init__()
        
        self.linear1 = nn.Linear(260, hidden_layer_size)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_layer_size, 130)
        self.logsoftmax = nn.LogSoftmax(dim=0)
        self.offsig = nn.Sigmoid()
        self.dursig = nn.Sigmoid()
    
    def forward(self, data):
        o = self.linear1(data)
        o = self.sigmoid(o)
        o = self.linear2(o)
        note_output = o[:,0:128]
        offset_output = o[:,128]
        duration_output = o[:,129]
        note_output = self.logsoftmax(note_output)
        offset_output = self.offsig(offset_output)
        duration_output = self.dursig(duration_output)
        return (note_output, offset_output, duration_output)

        

In [120]:
class MIDILSTMModel(nn.Module):
    def __init__(self, vocab_size=128, lstm_size=128, num_layers=3, dropout=0.2):
        super().__init__()
        
        self.lstm_size = lstm_size
        self.embeddings = nn.Embeddings(vocab_size, lstm_size)
        self.lstm = nn.LSTM(input_size=vocab_size, hidden_size=lstm_size, num_layers=num_layers, dropout=0.2)
        self.linear = nn.Linear(self.lstm_size, n_vocab)
        
    def forward(self, data, prev_state):
        emb = self.embeddings(data)
        output, state = self.lstm(data, prev_state)
        output = self.linear(output)
        return output, state
    
    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [15]:
def generate_samples_per_song(song):
    vectors = song.vector_encode()
    samples = []
    for i in range(2, len(vectors)):
        samples.append((torch.cat((vectors[i-2], vectors[i-1])), (torch.LongTensor([song.sequence[i][0]]), vectors[i][-2], vectors[i][-1])))
        
    return samples

def generate_samples(songlist):
    samples = []
    for song in songlist:
        samples += generate_samples_per_song(song)
        
    return samples

In [16]:
class MIDINotesDataset(Dataset):
    def __init__(self, mididir, maximum=500):
        items = os.walk(mididir)
        
        self.filenames = []
        for (directory, _, files) in items:
            self.filenames += [os.path.join(directory, x) for x in files]
        
        mss = []
        count = 0
        for x in self.filenames:
            try:
                midifile = MidiFile(x)
                ms = MessageSequence(midifile)
            except:
                continue
                
            mss.append(ms)
            
            count += 1
            if count == maximum:
                break
            
        self.notes = generate_samples(mss)
        
    def __getitem__(self, i):
        return self.notes[i]
    
    def __len__(self):
        return len(self.notes)

In [ ]:
class MIDISongsDataset(Dataset):
    def __init__(self, mididir, maximum=500):
        items = os.walk(mididir)
        
        self.filenames = []
        for (directory, _, files) in items:
            self.filenames += [os.path.join(directory, x) for x in files]
        
        mss = []
        count = 0
        for x in self.filenames:
            try:
                midifile = MidiFile(x)
                ms = MessageSequence(midifile)
            except:
                continue
                
            mss.append(ms)
            
            count += 1
            if count == maximum:
                break
                
        self.songs = [[x[0] for x in y.sequence] for y in mss]
        
    def

In [17]:
dataset = MIDINotesDataset("../clean_midi/", maximum=400)

In [18]:
len(dataset)

166422

In [19]:
dataset[50000:50010]

[(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000

In [20]:
dl = DataLoader(dataset, batch_size=25, shuffle=True)

In [21]:
dliter = iter(dl)

In [22]:
i, o = next(dliter)

In [23]:
i

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1600, 0.0040],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0295],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2400, 0.0275],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0950],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0800, 0.0020],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1480]])

In [24]:
i[:,0:128][3]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [25]:
def train(data, epochs=10):
    mm = MIDIModel()
    optimizer = optim.SGD(mm.parameters(), lr=0.001, momentum=0.9)
    note_criterion = nn.NLLLoss()
    for epoch in range(epochs):
        losses = []
        loader = DataLoader(data, batch_size=25, shuffle=True, drop_last=True)
        for i, o in loader:
            #print(i)
            #print(i.shape)
            #print(o)
            optimizer.zero_grad()
            (note_output, offset_output, duration_output) = mm(i)
            #print("no: {}, oo: {}, do: {}".format(note_output, offset_output, duration_output))
            #print(note_output, o[0].reshape((i.shape[0])))
            note_loss = torch.exp(-note_criterion(note_output, o[0].reshape((i.shape[0]))))
            offset_loss = torch.abs(o[1] - offset_output)
            duration_loss = torch.abs(o[2] - duration_output)
            #print("nl: {}, ol: {}, dl: {}".format(note_loss, offset_loss, duration_loss))
            loss = note_loss + offset_loss + duration_loss
            losses.append(sum(loss))
            sum(loss).backward()
            optimizer.step()
        print("mean loss in epoch {} is {}".format(epoch, float(torch.mean(torch.stack(losses)))))
    return mm

In [44]:
mm = MIDIModel()

In [45]:
samples = generate_samples([x])

In [46]:
samples

[(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000

In [26]:
model = train(dataset, epochs=10)

mean loss in epoch 0 is 9.282818794250488
mean loss in epoch 1 is 8.49507999420166
mean loss in epoch 2 is 8.014036178588867
mean loss in epoch 3 is 7.691254615783691
mean loss in epoch 4 is 7.602066516876221
mean loss in epoch 5 is 7.559309005737305
mean loss in epoch 6 is 7.530201435089111
mean loss in epoch 7 is 7.511342525482178
mean loss in epoch 8 is 7.4919633865356445
mean loss in epoch 9 is 7.472604751586914


In [48]:
samples[0][0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [113]:
from numpy.random import choice

# This is just to get the first two notes out of the development song.
vecs = x.vector_encode()

def generate_music(model, note1, note2, length=30, diversity=5):
    note_db = functional.one_hot(torch.arange(0, 128))
    newsong = [note1, note2]
    model.eval()
    with torch.no_grad():
        for i in range(length):
            notepair = torch.cat((note1, note2))
            fake_batch = torch.stack([notepair] + [torch.randn(260) for _ in range(24)])
            (note_output, offset_output, duration_output) = model(fake_batch)
            note_output = note_output[0]
            offset_output = offset_output[0]
            duration_output = duration_output[0]
            print("note_output: {}".format(note_output))
            notesort = torch.argsort(note_output, descending=True)
            print("notesort: {}".format(notesort))
            noteset = notesort[:diversity]
            print("noteset: {}".format(noteset))
            notenum = int(choice(noteset.numpy()))
            print("notenum: {}".format(notenum))
            note1 = note2
            print("testgen {} {} {}".format(note_db[notenum].clone().detach(), offset_output, duration_output))
            note2 = torch.cat((note_db[notenum].clone().detach(), torch.Tensor([offset_output]), 
                                                                               torch.Tensor([duration_output])))
            newsong.append(note2)
    return newsong

In [117]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0379, -0.0212, -0.0348,  ...,  0.0390, -0.6821,  0.2553],
         [ 0.0135, -0.0102,  0.0440,  ...,  0.0382, -0.4385, -0.1554],
         [-0.0459, -0.0201, -0.0308,  ...,  0.0471, -0.2274, -0.3345],
         ...,
         [ 0.0238, -0.0153,  0.0277,  ...,  0.0393, -0.5787,  0.2100],
         [-0.0466, -0.0477, -0.0502,  ...,  0.0445, -0.6573,  0.1680],
         [ 0.0166,  0.0107, -0.0525,  ...,  0.0313, -0.6278,  0.3004]],
        requires_grad=True),
 Parameter containing:
 tensor([-1.2920, -1.3207, -1.3000, -1.3046, -1.2779, -1.3535, -1.3260, -1.3103,
         -1.4531, -1.3560, -1.2794, -1.4405, -1.3082, -1.3466, -1.2668, -1.3131,
         -1.3471, -1.2969, -1.4687, -1.3851, -1.3341, -1.3543, -1.2969, -1.4141,
         -1.2835, -1.3050, -1.4415, -1.2941, -1.2931, -1.5521, -1.2762, -1.3198,
         -1.4111, -1.3167, -1.2821, -1.2745, -1.3009, -1.3128, -1.9857, -1.5490,
         -1.2962, -1.4991, -3.2269, -1.2598, -1.4787, -1.4079, -1.2939, -1.3072

In [114]:
newsong = generate_music(model, vecs[0], vecs[1], diversity=5)

note_output: tensor([ -3.1663,  -3.2132,  -3.2363,  -3.1139,  -3.1937,  -3.2603,  -3.2164,
         -3.2490,  -3.2749,  -3.2236,  -3.3018,  -3.2827,  -3.2674,  -3.2865,
         -3.1694,  -3.2225,  -3.3336,  -3.2446,  -3.0577,  -3.2212,  -3.3807,
         -3.1823,  -3.1949,  -3.1518,  -3.2189,  -4.2904,  -3.3889,  -3.7543,
         -5.0694,  -5.8594, -13.8011, -13.3104,  -7.1212, -11.7100,  -9.5984,
        -10.0661, -24.8011,  -8.5010, -12.6385,  -9.2809,  -6.7592,  -3.6233,
         -3.2653,  -3.5835,  -5.6451,  -3.0970,  -3.1823,  -4.0496,  -3.3581,
         -3.4113,  -9.1539,  -3.0823,  -4.8910,  -5.6605,  -3.1133, -12.0951,
         -3.0329, -11.2953,  -5.0951,  -9.5779, -16.1975,  -4.0627, -13.2139,
         -3.2412, -10.4758,  -8.6249,  -3.0369, -10.4775,  -3.2514,  -4.8874,
         -3.0329,  -3.8768,  -3.1982,  -4.3554,  -3.5411,  -3.4085,  -3.4552,
         -3.0919,  -3.2816,  -3.0537,  -3.0897,  -3.1140,  -3.0970,  -3.1125,
         -3.0850,  -3.1229,  -3.0877,  -3.3158,  -3

In [97]:
newsong

[tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [98]:
newsong[-1]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [99]:
torch.argmax(newsong[-1][0:128])


tensor(99)

In [100]:
p,q,r = model(torch.stack([torch.concat([vecs[0], vecs[1]])] + [torch.randn(260) for _ in range(24)]))
torch.argmax(p[0])

tensor(77)

In [101]:
def reconvert_song(notetensors):
    return [(int(torch.argmax(x[0:128])), int(torch.floor(x[128] * 100)), int(torch.floor(x[129] * 4000))) for x in notetensors]

In [115]:
converted_song = reconvert_song(newsong)

In [116]:
converted_song

[(59, 96, 96),
 (59, 96, 96),
 (79, 66, 219),
 (79, 68, 164),
 (56, 48, 119),
 (84, 13, 117),
 (3, 3, 71),
 (13, 0, 63),
 (76, 0, 66),
 (76, 0, 70),
 (43, 0, 65),
 (49, 1, 60),
 (76, 0, 75),
 (78, 0, 94),
 (13, 0, 81),
 (78, 0, 73),
 (26, 0, 87),
 (94, 1, 66),
 (3, 0, 68),
 (20, 0, 66),
 (41, 0, 64),
 (90, 2, 59),
 (49, 0, 70),
 (76, 0, 64),
 (76, 0, 93),
 (114, 0, 67),
 (3, 0, 68),
 (116, 0, 63),
 (78, 0, 67),
 (8, 0, 82),
 (114, 0, 69),
 (13, 0, 69)]

In [49]:
x.sequence

[(62, 4608, 54, 150),
 (64, 4704, 32, 224),
 (65, 4896, 54, 1494),
 (64, 6336, 66, 450),
 (60, 6720, 38, 230),
 (53, 6912, 40, 2344),
 (62, 9216, 24, 120),
 (64, 9312, 24, 216),
 (65, 9504, 40, 1480),
 (64, 10944, 48, 432),
 (60, 11328, 40, 232),
 (69, 11520, 40, 1192),
 (67, 12672, 28, 1180),
 (62, 13824, 60, 156),
 (64, 13920, 40, 232),
 (65, 14112, 56, 1496),
 (64, 15552, 22, 406),
 (60, 15936, 30, 222),
 (55, 16128, 44, 2348),
 (53, 18432, 24, 2136),
 (50, 20544, 34, 226),
 (53, 20736, 38, 1190),
 (52, 21888, 1114, 1152),
 (52, 32256, 38, 38),
 (62, 32256, 54, 150),
 (64, 32352, 40, 232),
 (65, 32544, 64, 1504),
 (64, 33984, 66, 450),
 (60, 34368, 34, 226),
 (53, 34560, 32, 2336),
 (62, 36864, 48, 144),
 (64, 36960, 48, 240),
 (65, 37152, 60, 1500),
 (64, 38592, 48, 432),
 (60, 38976, 40, 232),
 (69, 39168, 44, 1196),
 (67, 40320, 24, 1176),
 (62, 41472, 80, 176),
 (64, 41568, 40, 232),
 (65, 41760, 56, 1496),
 (64, 43200, 34, 418),
 (60, 43584, 30, 222),
 (55, 43776, 60, 2364),
 (

In [33]:
def get_sequence_back(model_ouptut, starting_time):
    sequence = []
    for (note, offset, duration) in model_ouptut:
        sequence.append((note, starting_time, offset, duration))
        starting_time += duration - offset
        
    return sequence

note1: note_on .................. note_off
note2:             note_on ................... note_off
note3:                      note_on ... note_off
note4:                                                   note_on .......... note_off


In [34]:
recovered_sequence = get_sequence_back(converted_song, 4608)

In [35]:
recovered_sequence

[(59, 4608, 96, 96),
 (59, 4608, 96, 96),
 (77, 4608, 66, 219),
 (66, 4761, 72, 165),
 (66, 4854, 24, 155),
 (42, 4985, 5, 179),
 (13, 5159, 5, 73),
 (47, 5227, 0, 67),
 (13, 5294, 0, 67),
 (49, 5361, 0, 57),
 (13, 5418, 1, 71),
 (49, 5488, 0, 90),
 (42, 5578, 1, 75),
 (13, 5652, 0, 64),
 (27, 5716, 0, 59),
 (13, 5775, 0, 59),
 (116, 5834, 0, 59),
 (3, 5893, 0, 69),
 (78, 5962, 0, 65),
 (26, 6027, 0, 82),
 (13, 6109, 1, 65),
 (3, 6173, 0, 65),
 (78, 6238, 0, 67),
 (12, 6305, 0, 82),
 (13, 6387, 0, 73),
 (26, 6460, 0, 67),
 (13, 6527, 0, 62),
 (13, 6589, 0, 64),
 (8, 6653, 0, 68),
 (3, 6721, 0, 66),
 (3, 6787, 0, 62),
 (78, 6849, 0, 63)]

In [20]:
x = os.walk("lakh/lmd_full")

In [21]:
x

<generator object _walk at 0x7f673fb7b140>

In [40]:
from scamp import *
import time

In [117]:
sess = Session().run_as_server()

In [118]:
clarinet = sess.new_part("clarinet")

Using preset Clarinet for clarinet


In [119]:
for n in converted_song:
    clarinet.play_note(n[0], 0.8, n[2]/1000)
    time.sleep(n[2]/1000 + 0.01)

In [57]:
t = torch.argsort(torch.Tensor([3, 4, 1, 2]), descending=True)

In [85]:
torch.Tensor([3, 4, 1, 2])[t[:3]][torch.multinomial(torch.Tensor([3, 4, 1, 2])[t[:3]], 1)]

tensor([2.])

In [87]:
torch.Tensor([3, 4, 1, 2]).numpy()

array([3., 4., 1., 2.], dtype=float32)